In [ ]:
!whereis pyspark

In [ ]:
pwd

In [ ]:
import findspark

In [ ]:
findspark.init("/usr/local/spark")

In [ ]:
import pyspark

In [ ]:
from pyspark import SparkContext

In [ ]:
sc = SparkContext.getOrCreate()

In [ ]:
print("Spark context", sc)

In [ ]:
x = sc.parallelize((10,20,30,40,50,60),2)

In [ ]:
type(x)

In [ ]:
x.getNumPartitions()

In [ ]:
x.collect()

In [ ]:
x.glom().collect()

In [ ]:
pwd

In [ ]:
# Transformations

In [ ]:
x_map = x.map(lambda i:i*i)

In [ ]:
x_map

In [ ]:
# Actions
x_map.collect()

In [ ]:
x_map.glom().collect()

In [ ]:
# Import a text file as a RDD

In [ ]:
lines = sc.textFile("customers.txt")

In [ ]:
help(sc.textFile)

In [ ]:
type(lines)

In [ ]:
lines.getNumPartitions()

In [ ]:
lines.collect()

In [ ]:
from pyspark.sql import Row

In [ ]:
fields = lines.map(lambda l:l.split('\t'))

In [ ]:
fields.collect()

In [ ]:
customers = fields.map(lambda p: Row(cid=p[0],c_name=p[1],c_city=p[2],c_state=p[3],c_zip=p[4]))

In [ ]:
customers.collect()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Python examples").getOrCreate()

In [ ]:
cust_df = spark.createDataFrame(customers)

In [ ]:
from pyspark.sql import *

In [ ]:
# Without Row function (Just pass the index)

In [ ]:
lines.collect()

In [ ]:
cfields = lines.map(lambda l:l.split('\t'))
custs = cfields.map(lambda p:(p[0],p[1],p[2],p[3],p[4]))

In [ ]:
custs.collect()

In [ ]:
cols = "cid cname ccity cstate zip"

In [ ]:
schema1 = [StructField(column_name, StringType(), True) for column_name in cols.split()]
schema_final = StructType(fields=schema1)

In [ ]:
df = spark.createDataFrame(custs, schema=schema_final)

In [ ]:
df.show(5)

In [ ]:
from pyspark.sql.types import StructField, StringType, IntegerType,StructType

In [ ]:
help(StructType)

In [ ]:
sfield = [StructField("cid", IntegerType(), True), StructField('cname', StringType(),True),StructField('ccity', StringType(),True),
         StructField('cstate', StringType(),True), StructField('zip', StringType(),True)]

In [ ]:
final_schema = StructType(fields=sfield)

In [ ]:
df1 = spark.read.csv('customers.txt',final_schema, sep='\t', header=True)

In [ ]:
df2 = spark.read.csv('customers.txt',inferSchema=True, sep='\t', header=True)

In [ ]:
df1.show(5)

In [ ]:
df2.show(5)

In [ ]:
df1.printSchema()

In [ ]:
df2.printSchema()

In [ ]:
cust_df.printSchema()

In [ ]:
customers.take(5)

In [ ]:
df2.columns

In [ ]:
df2.select('customer_city').show(5)

In [ ]:
df2.select(df2['customer_city'], df2["customer_state"]).show(5)

In [ ]:
df2.head(5)

In [ ]:
df2.filter(df2['customer_state'] == 'OH').show(5)

In [ ]:
df2.withColumn('double_age',df2['age']*2).show()

In [ ]:
df2.withColumnRenamed('customer_city','c_city').show()

In [ ]:
df2.groupBy('customer_state').count().show()

In [ ]:
import pandas

In [ ]:
df = df2.toPandas()

In [ ]:
# To use SQL queries on this data

In [ ]:
df2.createOrReplaceTempView("customers")

In [ ]:
statewise_analysis = spark.sql("SELECT customer_state, count(*) as statecount FROM customers  \
                             GROUP BY customer_state HAVING statecount>50")

In [ ]:
statewise_analysis.show(5)

In [ ]:
type(customers)

In [ ]:
statewise_analysis.printSchema()

In [ ]:
type(statewise_analysis)

In [ ]:
# Connect to Hive

In [ ]:
spark = SparkSession.builder.appName("HIve_integration") \
.config("spark.sql.warehouse.dir", "hdfs://localhost:54310/user/hive/warehouse")\
.enableHiveSupport() \
.getOrCreate()

In [ ]:
spark.sql("show databases").show()